# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 7 2022 11:31AM,238986,102,6554208,"Senseonics, Incorporated",Released
1,Jun 7 2022 11:25AM,238985,10,0085740852,ISDIN Corporation,Released
2,Jun 7 2022 11:25AM,238985,10,0085740872,ISDIN Corporation,Released
3,Jun 7 2022 11:25AM,238985,10,0085740961,ISDIN Corporation,Released
4,Jun 7 2022 11:25AM,238985,10,0085740960,ISDIN Corporation,Released
5,Jun 7 2022 11:25AM,238985,10,0085740962,ISDIN Corporation,Released
6,Jun 7 2022 11:25AM,238985,10,0085740966,ISDIN Corporation,Released
7,Jun 7 2022 11:25AM,238985,10,0085740996,ISDIN Corporation,Released
8,Jun 7 2022 11:25AM,238985,10,0085741005,ISDIN Corporation,Released
9,Jun 7 2022 11:25AM,238985,10,0085741006,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,238982,Released,1
41,238983,Released,1
42,238984,Released,2
43,238985,Released,43
44,238986,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238982,NaN,NaN,1.0
238983,NaN,NaN,1.0
238984,NaN,NaN,2.0
238985,NaN,NaN,43.0
238986,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238873,0.0,0.0,1.0
238879,0.0,41.0,3.0
238883,0.0,0.0,1.0
238888,0.0,0.0,1.0
238889,0.0,16.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238873,0,0,1
238879,0,41,3
238883,0,0,1
238888,0,0,1
238889,0,16,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238873,0,0,1
1,238879,0,41,3
2,238883,0,0,1
3,238888,0,0,1
4,238889,0,16,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238873,,,1
1,238879,,41,3
2,238883,,,1
3,238888,,,1
4,238889,,16,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 7 2022 11:31AM,238986,102,"Senseonics, Incorporated"
1,Jun 7 2022 11:25AM,238985,10,ISDIN Corporation
44,Jun 7 2022 11:15AM,238984,12,"NBTY Global, Inc."
46,Jun 7 2022 11:10AM,238983,10,"Methapharm, Inc."
47,Jun 7 2022 11:08AM,238943,15,"Person & Covey, Inc."
71,Jun 7 2022 11:03AM,238982,10,"Methapharm, Inc."
72,Jun 7 2022 10:58AM,238981,10,"Methapharm, Inc."
73,Jun 7 2022 10:42AM,238978,20,Else Nutrition
74,Jun 7 2022 10:27AM,238942,20,"ACI Healthcare USA, Inc."
91,Jun 7 2022 10:20AM,238926,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 7 2022 11:31AM,238986,102,"Senseonics, Incorporated",,,1
1,Jun 7 2022 11:25AM,238985,10,ISDIN Corporation,,,43
2,Jun 7 2022 11:15AM,238984,12,"NBTY Global, Inc.",,,2
3,Jun 7 2022 11:10AM,238983,10,"Methapharm, Inc.",,,1
4,Jun 7 2022 11:08AM,238943,15,"Person & Covey, Inc.",,,24
5,Jun 7 2022 11:03AM,238982,10,"Methapharm, Inc.",,,1
6,Jun 7 2022 10:58AM,238981,10,"Methapharm, Inc.",,,1
7,Jun 7 2022 10:42AM,238978,20,Else Nutrition,,,1
8,Jun 7 2022 10:27AM,238942,20,"ACI Healthcare USA, Inc.",,,17
9,Jun 7 2022 10:20AM,238926,20,"ACI Healthcare USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 11:31AM,238986,102,"Senseonics, Incorporated",1,,
1,Jun 7 2022 11:25AM,238985,10,ISDIN Corporation,43,,
2,Jun 7 2022 11:15AM,238984,12,"NBTY Global, Inc.",2,,
3,Jun 7 2022 11:10AM,238983,10,"Methapharm, Inc.",1,,
4,Jun 7 2022 11:08AM,238943,15,"Person & Covey, Inc.",24,,
5,Jun 7 2022 11:03AM,238982,10,"Methapharm, Inc.",1,,
6,Jun 7 2022 10:58AM,238981,10,"Methapharm, Inc.",1,,
7,Jun 7 2022 10:42AM,238978,20,Else Nutrition,1,,
8,Jun 7 2022 10:27AM,238942,20,"ACI Healthcare USA, Inc.",17,,
9,Jun 7 2022 10:20AM,238926,20,"ACI Healthcare USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 11:31AM,238986,102,"Senseonics, Incorporated",1,,
1,Jun 7 2022 11:25AM,238985,10,ISDIN Corporation,43,,
2,Jun 7 2022 11:15AM,238984,12,"NBTY Global, Inc.",2,,
3,Jun 7 2022 11:10AM,238983,10,"Methapharm, Inc.",1,,
4,Jun 7 2022 11:08AM,238943,15,"Person & Covey, Inc.",24,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 11:31AM,238986,102,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jun 7 2022 11:25AM,238985,10,ISDIN Corporation,43.0,NaN,NaN
2,Jun 7 2022 11:15AM,238984,12,"NBTY Global, Inc.",2.0,NaN,NaN
3,Jun 7 2022 11:10AM,238983,10,"Methapharm, Inc.",1.0,NaN,NaN
4,Jun 7 2022 11:08AM,238943,15,"Person & Covey, Inc.",24.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 11:31AM,238986,102,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jun 7 2022 11:25AM,238985,10,ISDIN Corporation,43.0,0.0,0.0
2,Jun 7 2022 11:15AM,238984,12,"NBTY Global, Inc.",2.0,0.0,0.0
3,Jun 7 2022 11:10AM,238983,10,"Methapharm, Inc.",1.0,0.0,0.0
4,Jun 7 2022 11:08AM,238943,15,"Person & Covey, Inc.",24.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2628548,56.0,9.0,5.0
102,238986,1.0,0.0,0.0
12,238984,2.0,0.0,0.0
15,1194707,106.0,1.0,0.0
16,477814,0.0,26.0,0.0
17,477784,2.0,0.0,0.0
18,716733,3.0,0.0,0.0
19,955713,4.0,1.0,0.0
20,2150359,56.0,108.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2628548,56.0,9.0,5.0
1,102,238986,1.0,0.0,0.0
2,12,238984,2.0,0.0,0.0
3,15,1194707,106.0,1.0,0.0
4,16,477814,0.0,26.0,0.0
5,17,477784,2.0,0.0,0.0
6,18,716733,3.0,0.0,0.0
7,19,955713,4.0,1.0,0.0
8,20,2150359,56.0,108.0,0.0
9,21,238916,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,56.0,9.0,5.0
1,102,1.0,0.0,0.0
2,12,2.0,0.0,0.0
3,15,106.0,1.0,0.0
4,16,0.0,26.0,0.0
5,17,2.0,0.0,0.0
6,18,3.0,0.0,0.0
7,19,4.0,1.0,0.0
8,20,56.0,108.0,0.0
9,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,56.0
1,102,Released,1.0
2,12,Released,2.0
3,15,Released,106.0
4,16,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,18,19,20,21
Status,,,,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,0.0,0.0,1.0,26.0,0.0,0.0,1.0,108.0,0.0
Released,56.0,1.0,2.0,106.0,0.0,2.0,3.0,4.0,56.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,18,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,0.0,1.0,26.0,0.0,0.0,1.0,108.0,0.0
2,Released,56.0,1.0,2.0,106.0,0.0,2.0,3.0,4.0,56.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,18,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,0.0,1.0,26.0,0.0,0.0,1.0,108.0,0.0
2,Released,56.0,1.0,2.0,106.0,0.0,2.0,3.0,4.0,56.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()